##### Quick notes on the concept which were not in Deep Learning modules of Machine Learning Nano Degree:

##### Regarding DL for Text :
* In text dataset, key terms are more important and we might no have enough dataset describing them. Deep Learning model usually expects many samples to understand the variance of a label. like image of a perticular flower.
* Semantic relation between kitty and cat understanding also requires huge dataset for DL. If 2 words are similar then we have to share weights between them. i.e. the model should predict they are same when asked so the the internal weights of the model should indicate that the weights values are such that out of so huge NN (CNN) the trajectory of the word transformation for both the words are same.

##### similar words occur in similar context:
* As we have lot of text from the the wikipedia, we can have unsupervised learning to identify the similar words with **the assumption that similar words occur in similar context.** 
* This assumption helps in 2 ways :
    * without knowing their meaning we can have associated words.
    * representation of words can be done through the associated words rather than sparse matrix of n words with n words. This representation of words by associated word vector is called **embedding**. Clearly, this is smaller vector from the sparse matrix.
    * with 1 word embedding we have multiple words association which are in the word embedding is obtained.
    
##### How do we get the word embedding?
* **Word2Vec**: split a sentence it to 2 sets or words. One set of words trying to predict other set of words. Here simple logistic regression can be used to generate the model for prediction. The best set of word (also called window of words) that best represents the corpus (sentences, paragraph or book) is then saved.
    * **skip grams**: predicts the neighboring context from a word
    * **Continuous Bag Of Words**: predicts the word from neighboring context.
    * https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/
    * **Sampled Softmax**: When vocabulary of the target words are large, softmax may be in-efficient. Therefore, **non class targets** are randomly sampled and with this small subset used for training the model.

* **t-SNE**: is a visualization technique where the association of words are projected onto 2D with relation between the words are retained. Closer words are plotted closer and loosely coupled words are plotted far away.

##### So, now, we have word embedding...what next ?
* compare them using cosine similarity. Why compare? thats the most common activity on text. Applications like plagarism check, classifying documents based on authors require word embedding to be compared.


* can it also help in Q&A ? predicting next sequence of sentences ?

##### RNN: Recurring Neural Network
* takes into consideration of input vector time. i.e. what was the state of input at t-1th instance. it shares weights across time rather than space as in CNN.
* since input changes over time and at any point it depends on previous state, a model is used to summarized the events at past. This model forms the recurrent connetion.
* A repeatable network of recurrent connection to summarize the past, input vector and predicting model (classification or regression) is called **RNN**. Usually the predicting model is same in the repeated network.
* Here input vector remains the same but the tth second we would take into consider recursive transformation over t-1 and before.
* **back propagation over time** would result in correlated update to weights which would result in bad gradient descent. Correlated update is because of recurrent network which has the wieght carried in the output of t-1 second.

* **vanishing gradients**: here gradients abruptly decreases to zero. This would result no training to model. Also result in memory loss and model do not keep tracks of more distant past and keeps track of only recent past.

* **exploding gradients**: here gradients abruptly increases to infinity. This can be addressed by Gradient clipping where gradients are normalized and when gradients are too huge the steps are cut shorted. i.e. when u start back propagating the point at which it reaches the max threshold u just stop increasing the gradients and the same value is assumed for t-x and less.


##### LSTM: Long Short Term Memory
* addresses the vanishing gradients problem in RNN
* replaces the predicting model in RNN with LSTM "cell" which does the memory management. This memory management helps network to remember the older events. Thus solving vanishing gradient problem.
* memory management operation involves storing, forgetting and reading memory.
* instead of making those operation descrete result oriented, it is controlled to through logistic regression which makes it easier for partial read, store and delete.
* **regularization**: L2 regularization can be used on all 4 sides (2 inputs and 2 outputs). Dropout can be used in only input and/or output and not between recurring network and connect future areas.

##### RNN Application : Beam Search
* RNN are for time series activities prediction, one example is word prediction like that one Google search bar does.
* It takes the input of so far entered text and predicts the upcoming word or characters.
* Taking one prediction at a time would be too greedy, rather to take couple of them and continue the sequence for all of them.
* this would mean that there are now multiple sequence being investigated 
* Now based on the final probabilities score, one sequence can be finalized
* This approach is good to avoid scenario which accidentally landing to one word, but now we have couple sequences to choose from.

e.g.: App can lead to both apple or append or application. Now just because one model chooses,say apple, because its probability is slightly larger than other you would loose option of other 2 words.

If you had chosen both 'l' & 'e' and continued series then you would have at the end to choose from apple, append and application. Now based on the probability score (multiply all the predicted probabilities) you are in better position to choose the word.

* Maintaining all sequences might not be cost effective(time and memory). For this we have Beam search.
* it prunes the sequence which are not so likely and keeps only the most likely. It define a beam width which keeps the most 